<h2 align='center'>Codebasics ML Course: ML Flow Dagshub Tutorial</h2>

In [1]:
# pip uninstall -y numpy

In [2]:
# pip cache purge   # clear pip cache

In [3]:
# pip install numpy==1.26.4

In [4]:
import numpy as np
print(np.__version__)

1.26.4


In [5]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [6]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.8, 0.2], flip_y=0, random_state=2)

np.unique(y, return_counts=True)

(array([0, 1]), array([800, 200]))

In [7]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=2)

#### Handle class imbalance

In [8]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([620, 620]))

### Track Experiments

In [9]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [10]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [11]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

# **DagsHub Demo**

In [12]:
# %pip install -q dagshub 'mlflow>=2,<3'

In [15]:
import dagshub

In [16]:
# dagshub setup
# for first time, Run this cell and give access to your dagshub with limited validity

# dagshub.init(repo_owner='your_dagshub_username', repo_name='your_dagshub_repo', mlflow=True)

In [ ]:
# Initialize MLflow
mlflow.set_experiment("Anomaly Detection")
# mlflow.set_tracking_uri("http://localhost:5000")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2025/08/19 12:10:44 INFO mlflow.tracking.fluent: Experiment with name 'Anomaly Detection' does not exist. Creating a new experiment.
2025/08/19 12:10:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression at: https://dagshub.com/arpitpatelsitapur/my-first-repo.mlflow/#/experiments/0/runs/a333810733194610ae321be076686eef
🧪 View experiment at: https://dagshub.com/arpitpatelsitapur/my-first-repo.mlflow/#/experiments/0


2025/08/19 12:11:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: https://dagshub.com/arpitpatelsitapur/my-first-repo.mlflow/#/experiments/0/runs/80ee5367bf704162aa543fb35ba6a73d
🧪 View experiment at: https://dagshub.com/arpitpatelsitapur/my-first-repo.mlflow/#/experiments/0


2025/08/19 12:11:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier at: https://dagshub.com/arpitpatelsitapur/my-first-repo.mlflow/#/experiments/0/runs/97f33ab269c64c478b53ad2370590189
🧪 View experiment at: https://dagshub.com/arpitpatelsitapur/my-first-repo.mlflow/#/experiments/0


2025/08/19 12:11:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: https://dagshub.com/arpitpatelsitapur/my-first-repo.mlflow/#/experiments/0/runs/9cf9d29ed6a745b29eb6ff4e6ba65e72
🧪 View experiment at: https://dagshub.com/arpitpatelsitapur/my-first-repo.mlflow/#/experiments/0


After running above cell, if u visit your dagshub repo, u will see something like this -

![Dagshub](dagshub_exp_tracking.png)